# Aula 10 - Pré-processamento de Dados: Transformação e Normalização

**Objetivo:** Aprender técnicas de transformação de dados para análise exploratória avançada

**Dataset:** Telco Customer Churn (previsão de cancelamento de clientes)

---

## Agenda

1. **Feature Engineering** - Criar variáveis derivadas úteis
2. **Encoding** - Converter categorias em números
3. **Normalização** - Ajustar escalas para comparabilidade
4. **Pipeline Completo** - Integrar todas as transformações

---

## Setup Inicial

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas com sucesso!")

## Carregar Dataset

In [ ]:
# Carregar dados
url = 'https://raw.githubusercontent.com/marvin-rubia/Churn-Analysis-Prediction/main/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(url)


print("=== INFORMAÇÕES DO DATASET ===")
print(f"Shape: {df.shape}")
print(f"\nColunas: {df.columns.tolist()}")
print(f"\nTipos de dados:")
print(df.dtypes.value_counts())

# Primeiras linhas
df.head()

## Entendimento Rápido dos Dados

In [ ]:
# Estatísticas descritivas - numéricas
print("=== VARIÁVEIS NUMÉRICAS ===")
df.describe()

In [ ]:
# Estatísticas descritivas - categóricas
print("=== VARIÁVEIS CATEGÓRICAS ===")
df.describe(include='object')

In [ ]:
# Distribuição do target (Churn)
print("=== DISTRIBUIÇÃO DO TARGET (CHURN) ===")
print(df['Churn'].value_counts())
print(f"\nTaxa de Churn: {(df['Churn'] == 'Yes').mean()*100:.2f}%")

# Visualizar
fig, ax = plt.subplots(figsize=(8, 5))
df['Churn'].value_counts().plot(kind='bar', ax=ax, color=['skyblue', 'salmon'])
ax.set_title('Distribuição de Churn', fontsize=14)
ax.set_xlabel('Churn')
ax.set_ylabel('Quantidade')
ax.set_xticklabels(['No', 'Yes'], rotation=0)
plt.tight_layout()
plt.show()

## Tratamento Básico

Antes de começar as transformações, vamos fazer alguns tratamentos básicos.

In [ ]:
# Converter TotalCharges para numérico (pode ter strings vazias)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Verificar missing values
print("=== MISSING VALUES ===")
missing = df.isnull().sum()
print(missing[missing > 0])

# Preencher TotalCharges missing com MonthlyCharges (clientes novos)
df['TotalCharges'].fillna(df['MonthlyCharges'], inplace=True)

print("\n✓ Tratamento básico completo!")

---

# BLOCO 1: Feature Engineering

**Objetivo:** Criar novas variáveis (features) a partir das existentes para revelar padrões ocultos.

**Estratégia:**
1. Criar feature
2. Validar através de análise exploratória
3. Verificar correlação com Churn
4. Decidir se mantém ou descarta

---

## Técnica 1: Operações Aritméticas Simples

**Feature:** Gasto médio mensal (AvgMonthlySpend)

**Raciocínio:** TotalCharges / tenure revela padrão de gasto mais claramente que valores absolutos

In [ ]:
# Criar feature
df['AvgMonthlySpend'] = df['TotalCharges'] / df['tenure']

# Tratar casos especiais (tenure = 0)
df['AvgMonthlySpend'] = df['AvgMonthlySpend'].replace([np.inf, -np.inf], np.nan)
df.loc[df['tenure'] == 0, 'AvgMonthlySpend'] = df.loc[df['tenure'] == 0, 'MonthlyCharges']

print("=== FEATURE CRIADA: AvgMonthlySpend ===")
print(df['AvgMonthlySpend'].describe())

In [ ]:
# Validação: Comparar por Churn
print("=== COMPARAÇÃO POR CHURN ===")
comparison = df.groupby('Churn')['AvgMonthlySpend'].describe()
print(comparison)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot
sns.boxplot(data=df, x='Churn', y='AvgMonthlySpend', ax=axes[0])
axes[0].set_title('Gasto Médio Mensal por Churn')
axes[0].set_ylabel('Gasto Médio Mensal (R$)')

# Histogramas sobrepostos
df[df['Churn']=='No']['AvgMonthlySpend'].hist(bins=30, alpha=0.5, label='No Churn', ax=axes[1], color='skyblue')
df[df['Churn']=='Yes']['AvgMonthlySpend'].hist(bins=30, alpha=0.5, label='Churn', ax=axes[1], color='salmon')
axes[1].set_xlabel('Gasto Médio Mensal')
axes[1].set_title('Distribuição por Grupo')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n✓ Clientes com churn têm gasto médio mensal ligeiramente maior")

## Técnica 2: Diferenças entre Variáveis

**Feature:** Diferença entre gasto atual e média histórica (ChargeDiff)

**Raciocínio:** Aumento recente no preço pode indicar insatisfação

In [ ]:
# Criar feature
df['ChargeDiff'] = df['MonthlyCharges'] - df['AvgMonthlySpend']

print("=== FEATURE CRIADA: ChargeDiff ===")
print(df['ChargeDiff'].describe())

print("\n=== INTERPRETAÇÃO ===")
print("ChargeDiff > 0: Gasto atual maior que média (possível aumento recente)")
print("ChargeDiff < 0: Gasto atual menor que média (possível desconto ou redução)")
print("ChargeDiff ≈ 0: Gasto estável")

# Analisar por churn
print("\n=== MÉDIA POR CHURN ===")
print(df.groupby('Churn')['ChargeDiff'].mean())

## Técnica 3: Razões e Proporções

**Feature:** Proporção do tempo como cliente (TenureRatio)

**Raciocínio:** Normalizar tenure facilita interpretação e comparação

In [ ]:
# Criar feature
max_tenure = df['tenure'].max()
df['TenureRatio'] = df['tenure'] / max_tenure

print("=== FEATURE CRIADA: TenureRatio ===")
print(f"Range: [{df['TenureRatio'].min():.2f}, {df['TenureRatio'].max():.2f}]")
print(df['TenureRatio'].describe())

# Criar categorias baseadas no ratio
df['TenureCategory'] = pd.cut(df['TenureRatio'],
                               bins=[0, 0.33, 0.66, 1.0],
                               labels=['Novo', 'Médio', 'Veterano'])

print("\n=== DISTRIBUIÇÃO POR CATEGORIA ===")
print(pd.crosstab(df['TenureCategory'], df['Churn'], normalize='index'))

## Técnica 4: Binning (Discretização)

**Feature:** MonthlyCharges em faixas de preço

**Raciocínio:** Agrupar em faixas facilita análise e visualização de padrões

In [ ]:
# Método 1: Bins com larguras iguais
df['ChargesBin_Equal'] = pd.cut(df['MonthlyCharges'],
                                 bins=5,
                                 labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto'])

# Método 2: Quantis (mesma quantidade em cada bin)
df['ChargesBin_Quantile'] = pd.qcut(df['MonthlyCharges'],
                                     q=5,
                                     labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'],
                                     duplicates='drop')

print("=== BINNING CRIADO ===")
print("\nDistribuição - Bins Iguais:")
print(df['ChargesBin_Equal'].value_counts().sort_index())

print("\nDistribuição - Quantis:")
print(df['ChargesBin_Quantile'].value_counts().sort_index())

In [ ]:
# Analisar churn por faixa de preço
print("=== CHURN RATE POR FAIXA ===")
churn_by_bin = df.groupby('ChargesBin_Equal')['Churn'].apply(
    lambda x: (x == 'Yes').mean()
)
print(churn_by_bin)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 6))
churn_by_bin.plot(kind='bar', color='coral', ax=ax)
ax.set_title('Taxa de Churn por Faixa de Preço', fontsize=14)
ax.set_ylabel('Taxa de Churn')
ax.set_xlabel('Faixa de MonthlyCharges')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.axhline(y=(df['Churn']=='Yes').mean(), color='red', linestyle='--', alpha=0.5, label='Média geral')
ax.legend()
plt.tight_layout()
plt.show()

print("\n✓ Churn aumenta significativamente em faixas de preço mais alto")

## Técnica 5: Agregação - Contar Serviços

**Feature:** Total de serviços adicionais (TotalServices)

**Raciocínio:** Clientes com mais serviços podem estar mais "engajados"

In [ ]:
# Lista de colunas de serviços
service_cols = [
    'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Contar serviços ativos
df['TotalServices'] = 0
for col in service_cols:
    # Adiciona 1 se não for "No" ou "No internet service" ou "No phone service"
    df['TotalServices'] += (~df[col].isin(['No', 'No internet service', 'No phone service'])).astype(int)

print("=== FEATURE CRIADA: TotalServices ===")
print(df['TotalServices'].describe())

print("\n=== DISTRIBUIÇÃO ===")
print(df['TotalServices'].value_counts().sort_index())

In [ ]:
# Analisar relação com Churn
print("=== MÉDIA DE SERVIÇOS POR CHURN ===")
print(df.groupby('Churn')['TotalServices'].mean())

# Visualizar
fig, ax = plt.subplots(figsize=(12, 6))
pd.crosstab(df['TotalServices'], df['Churn']).plot(kind='bar', ax=ax)
ax.set_title('Distribuição de Churn por Total de Serviços', fontsize=14)
ax.set_xlabel('Número de Serviços')
ax.set_ylabel('Quantidade de Clientes')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.legend(['No Churn', 'Churn'])
plt.tight_layout()
plt.show()

print("\n✓ Interessante: parece haver padrão em U - baixo e alto número de serviços associados a mais churn")

## Técnica 6: Flags Booleanas

**Features:** Indicadores binários (0/1) para condições específicas

**Raciocínio:** Simplificar análise de combinações importantes

In [ ]:
# Criar flags
df['HasInternet'] = (df['InternetService'] != 'No').astype(int)
df['HasPhone'] = (df['PhoneService'] == 'Yes').astype(int)
df['SeniorWithDependents'] = ((df['SeniorCitizen'] == 1) & (df['Dependents'] == 'Yes')).astype(int)
df['IsPremium'] = ((df['TotalServices'] >= 6) & (df['Contract'].isin(['One year', 'Two year']))).astype(int)

print("=== FLAGS CRIADAS ===")
flags = ['HasInternet', 'HasPhone', 'SeniorWithDependents', 'IsPremium']

for flag in flags:
    count = df[flag].sum()
    pct = df[flag].mean() * 100
    print(f"{flag}: {count} ({pct:.1f}%)")

In [ ]:
# Analisar impacto no Churn
print("=== ANÁLISE DE FLAGS vs CHURN ===\n")

for flag in flags:
    print(f"\n--- {flag} ---")

    # Taxa de churn por flag
    churn_rate = df.groupby(flag)['Churn'].apply(lambda x: (x == 'Yes').mean())
    print("Taxa de Churn:")
    print(churn_rate)

    # Diferença
    if len(churn_rate) == 2:
        diff = churn_rate[1] - churn_rate[0]
        print(f"Diferença: {diff*100:.1f} pontos percentuais")

## Técnica 7: Interações entre Variáveis

**Features:** Combinações que capturam efeitos sinérgicos

**Raciocínio:** Algumas combinações têm efeito maior que variáveis isoladas

In [ ]:
# Criar interações
# Interação 1: Contrato curto + Preço alto (hipótese: combinação perigosa)
df['ShortContract_HighPrice'] = (
    (df['Contract'] == 'Month-to-month') &
    (df['MonthlyCharges'] > df['MonthlyCharges'].quantile(0.75))
).astype(int)

# Interação 2: Idoso sem segurança online
df['Senior_NoSecurity'] = (
    (df['SeniorCitizen'] == 1) &
    (df['OnlineSecurity'] == 'No')
).astype(int)

# Interação 3: Fibra + Streaming (usuário power)
df['FiberStreamer'] = (
    (df['InternetService'] == 'Fiber optic') &
    ((df['StreamingTV'] == 'Yes') | (df['StreamingMovies'] == 'Yes'))
).astype(int)

print("=== INTERAÇÕES CRIADAS ===")
interactions = ['ShortContract_HighPrice', 'Senior_NoSecurity', 'FiberStreamer']

for inter in interactions:
    count = df[inter].sum()
    pct = df[inter].mean() * 100
    print(f"{inter}: {count} ({pct:.1f}%)")

In [ ]:
# Analisar impacto das interações
print("=== ANÁLISE DE INTERAÇÕES ===\n")

for interaction in interactions:
    print(f"\n--- {interaction} ---")

    # Crosstab com Churn
    ct = pd.crosstab(df[interaction], df['Churn'], normalize='index')
    print(ct)

    # Taxa de churn quando interação = 1
    churn_rate_with = df[df[interaction] == 1]['Churn'].apply(lambda x: 1 if x == 'Yes' else 0).mean()
    overall_rate = (df['Churn'] == 'Yes').mean()

    print(f"\nChurn Rate com {interaction}=1: {churn_rate_with*100:.1f}%")
    print(f"Churn Rate geral: {overall_rate*100:.1f}%")
    print(f"Diferença: {(churn_rate_with - overall_rate)*100:.1f} pontos percentuais")

## Técnica 8: Percentis e Rankings

**Feature:** Posição relativa no dataset

**Raciocínio:** Às vezes importa mais a posição relativa que o valor absoluto

In [ ]:
# Criar features de ranking
df['TotalCharges_Percentile'] = df['TotalCharges'].rank(pct=True) * 100
df['TotalCharges_Rank'] = df['TotalCharges'].rank(ascending=False)

# Categorizar por percentil
df['ChargesLevel'] = pd.cut(df['TotalCharges_Percentile'],
                             bins=[0, 25, 50, 75, 100],
                             labels=['Bottom 25%', 'Q2', 'Q3', 'Top 25%'])

print("=== DISTRIBUIÇÃO POR NÍVEL ===")
print(df['ChargesLevel'].value_counts())

# Analisar churn por nível
print("\n=== CHURN POR NÍVEL ===")
print(pd.crosstab(df['ChargesLevel'], df['Churn'], normalize='index'))

## Técnica 9: Z-Score para Detecção de Anomalias

**Feature:** Identificar valores extremos (outliers)

**Raciocínio:** Clientes com comportamento atípico podem ter padrão diferente de churn

In [ ]:
# Calcular Z-scores
df['MonthlyCharges_Zscore'] = stats.zscore(df['MonthlyCharges'])

# Flag para outliers (|Z| > 3)
df['IsOutlier_Charges'] = (abs(df['MonthlyCharges_Zscore']) > 3).astype(int)

print("=== OUTLIERS IDENTIFICADOS ===")
print(f"Total de outliers: {df['IsOutlier_Charges'].sum()}")
print(f"Percentual: {df['IsOutlier_Charges'].mean()*100:.2f}%")

# Analisar outliers
print("\n=== CHURN ENTRE OUTLIERS ===")
print(pd.crosstab(df['IsOutlier_Charges'], df['Churn'], normalize='index'))

## Técnica 10: Features Baseadas em Conhecimento do Domínio

**Features:** Perfis de risco baseados em expertise

**Raciocínio:** Combinar múltiplos fatores que especialistas identificam como importantes

In [ ]:
# Feature: High Risk Profile
df['HighRiskProfile'] = (
    (df['Contract'] == 'Month-to-month') &  # Sem compromisso
    (df['tenure'] < 12) &                    # Cliente novo
    (df['TotalServices'] <= 2) &             # Poucos serviços
    (df['PaymentMethod'] == 'Electronic check')  # Método menos estável
).astype(int)

# Feature: Value Customer
df['ValueCustomer'] = (
    (df['Contract'].isin(['One year', 'Two year'])) &  # Contrato longo
    (df['TotalServices'] >= 4) &                       # Muitos serviços
    (df['tenure'] > 24) &                              # Cliente antigo
    (df['MonthlyCharges'] > df['MonthlyCharges'].median())  # Gasto alto
).astype(int)

print("=== PERFIS CRIADOS ===")
profiles = ['HighRiskProfile', 'ValueCustomer']

for profile in profiles:
    count = df[profile].sum()
    pct = df[profile].mean() * 100
    print(f"{profile}: {count} ({pct:.1f}%)")

In [ ]:
# Validar perfis
print("=== ANÁLISE DE PERFIS ===\n")

for profile in profiles:
    print(f"\n--- {profile} ---")

    # Churn rate
    churn_rate = df[df[profile] == 1]['Churn'].apply(lambda x: 1 if x == 'Yes' else 0).mean()
    baseline = (df['Churn'] == 'Yes').mean()

    print(f"Churn Rate: {churn_rate*100:.1f}%")
    print(f"Baseline: {baseline*100:.1f}%")
    print(f"Diferença: {(churn_rate - baseline)*100:.1f} pontos percentuais")

## Resumo: Features Criadas

Vamos listar todas as features que criamos e suas correlações com Churn.

In [ ]:
# Listar features criadas
created_features = [
    'AvgMonthlySpend', 'ChargeDiff', 'TenureRatio',
    'TotalServices', 'HasInternet', 'HasPhone',
    'SeniorWithDependents', 'IsPremium',
    'ShortContract_HighPrice', 'Senior_NoSecurity', 'FiberStreamer',
    'TotalCharges_Percentile', 'IsOutlier_Charges',
    'HighRiskProfile', 'ValueCustomer'
]

print(f"=== TOTAL DE FEATURES CRIADAS: {len(created_features)} ===")
print("\nFeatures:")
for feat in created_features:
    print(f"  - {feat}")

In [ ]:
# Criar target binário para correlações
df['Churn_binary'] = (df['Churn'] == 'Yes').astype(int)

# Calcular correlações com Churn
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
correlations = df[numeric_features].corr()['Churn_binary'].drop('Churn_binary')
correlations_abs = correlations.abs().sort_values(ascending=False)

print("=== TOP 15 FEATURES MAIS CORRELACIONADAS COM CHURN ===")
print(correlations_abs.head(15))

In [ ]:
# Visualizar top correlações
fig, ax = plt.subplots(figsize=(10, 8))

top_20 = correlations_abs.head(20)
top_20.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Correlação Absoluta com Churn')
ax.set_title('Top 20 Features Mais Correlacionadas com Churn', fontsize=14)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n✓ Feature Engineering completo!")
print("✓ Próximo: Encoding de variáveis categóricas")

---

**Checkpoint:** Salvar progresso antes de continuar para o Encoding.

In [ ]:
# Salvar dataset com features criadas
df.to_csv('telco_churn_with_features.csv', index=False)
print("✓ Dataset salvo: telco_churn_with_features.csv")
print(f"Shape atual: {df.shape}")

# Aula 10 - PARTE 2: Encoding e Normalização

**Continuação da transformação de dados**

Nesta parte:
- **Bloco 2:** Encoding de variáveis categóricas
- **Bloco 3:** Normalização e padronização

---

## Setup (caso esteja começando daqui)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

print("✓ Setup completo!")

In [ ]:
# Carregar dados (com features criadas na Parte 1)
df = pd.read_csv('telco_churn_with_features.csv')

print(f"Shape: {df.shape}")
print(f"\n✓ Dataset carregado com {df.shape[1]} colunas")

---

# BLOCO 2: Encoding de Variáveis Categóricas

**Objetivo:** Converter todas as variáveis categóricas em numéricas para permitir análises quantitativas.

**Estratégias:**
1. **Label Encoding** - Para ordinais (variáveis com ordem natural)
2. **One-Hot Encoding** - Para nominais (variáveis sem ordem)
3. **Binary Encoding** - Para binárias (Yes/No, True/False)

---

## Identificar Tipos de Variáveis Categóricas

Primeiro, vamos classificar nossas variáveis categóricas por tipo.

In [ ]:
# Listar variáveis categóricas
categorical_cols = df.select_dtypes(include='object').columns.tolist()

# Remover IDs e target original
categorical_cols = [c for c in categorical_cols if c not in ['customerID', 'Churn']]

print("=== VARIÁVEIS CATEGÓRICAS PARA ENCODING ===")
print(f"\nTotal: {len(categorical_cols)}\n")

for col in categorical_cols:
    unique_vals = df[col].nunique()
    print(f"{col}: {unique_vals} valores únicos")
    print(f"  Valores: {df[col].unique()[:5].tolist()}")
    print()

## Classificação Manual das Variáveis

In [ ]:
# Classificar por tipo
ordinal_vars = {
    'Contract': {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
}

binary_vars = [
    'gender', 'Partner', 'Dependents',
    'PhoneService', 'PaperlessBilling'
]

nominal_vars = [
    'PaymentMethod', 'InternetService'
]

# Variáveis com "No service" (tratamento especial)
service_with_no = [
    'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

print("=== CLASSIFICAÇÃO ===")
print(f"\nOrdinais: {len(ordinal_vars)}")
print(f"Binárias: {len(binary_vars)}")
print(f"Nominais: {len(nominal_vars)}")
print(f"Serviços (com 'No service'): {len(service_with_no)}")

## 1. Label Encoding - Variáveis Ordinais

**Contract** é ordinal porque tem ordem natural de compromisso: Month-to-month < One year < Two year

In [ ]:
# Label Encoding manual (preserva ordem)
print("=== LABEL ENCODING: Contract ===")

contract_mapping = ordinal_vars['Contract']
df['Contract_encoded'] = df['Contract'].map(contract_mapping)

print("\nMapeamento:")
for category, code in contract_mapping.items():
    print(f"  {category} → {code}")

print("\nVerificação:")
print(df[['Contract', 'Contract_encoded']].drop_duplicates().sort_values('Contract_encoded'))

In [ ]:
# Validar: correlação com Churn
from scipy.stats import pearsonr

corr, pval = pearsonr(df['Contract_encoded'], df['Churn_binary'])
print("=== VALIDAÇÃO: Contract_encoded vs Churn ===")
print(f"Correlação: {corr:.3f} (p-value: {pval:.4f})")

# Visualizar
churn_by_contract = df.groupby('Contract_encoded')['Churn'].apply(lambda x: (x == 'Yes').mean())

fig, ax = plt.subplots(figsize=(8, 5))
churn_by_contract.plot(kind='bar', color='salmon', ax=ax)
ax.set_title('Taxa de Churn por Tipo de Contrato', fontsize=14)
ax.set_xlabel('Contract (0=Month, 1=1yr, 2=2yr)')
ax.set_ylabel('Taxa de Churn')
ax.set_xticklabels(['Month-to-month', 'One year', 'Two year'], rotation=45)
plt.tight_layout()
plt.show()

print("\n✓ Correlação negativa forte: contratos mais longos → menos churn")

## 2. Binary Encoding - Variáveis Binárias (Yes/No)

Para variáveis com apenas 2 valores, usamos 0 e 1.

In [ ]:
# Encoding de binárias
print("=== BINARY ENCODING ===")

for col in binary_vars:
    # Método 1: Operador booleano (mais conciso)
    df[f'{col}_encoded'] = (df[col] == 'Yes').astype(int)
    # Para gender: Male=1, Female=0
    if col == 'gender':
        df[f'{col}_encoded'] = (df[col] == 'Male').astype(int)

    print(f"✓ {col} encoded")

# SeniorCitizen já é 0/1
df['SeniorCitizen_encoded'] = df['SeniorCitizen']
print("✓ SeniorCitizen (já numérico)")

In [ ]:
# Verificar encodings
print("=== VERIFICAÇÃO DOS ENCODINGS ===")

for col in binary_vars:
    encoded_col = f'{col}_encoded'
    print(f"\n{col}:")
    print(df[[col, encoded_col]].drop_duplicates().sort_values(encoded_col))

## 3. Tratamento Especial - Variáveis com "No Service"

Algumas variáveis têm 3 valores: Yes, No, No internet/phone service.

Vamos criar variável binária: tem o serviço (Yes) ou não tem (No ou No service).

In [ ]:
# Encoding de serviços
print("=== ENCODING: Variáveis de Serviço ===")

for col in service_with_no:
    # 1 se 'Yes', 0 caso contrário
    df[f'{col}_encoded'] = (df[col] == 'Yes').astype(int)
    print(f"✓ {col} encoded")

print("\n✓ Todas as variáveis de serviço encodadas como binárias")

In [ ]:
# Exemplo de verificação
print("=== EXEMPLO: OnlineSecurity ===")
print(df[['OnlineSecurity', 'OnlineSecurity_encoded']].value_counts().sort_index())

## 4. One-Hot Encoding - Variáveis Nominais

Para variáveis sem ordem natural, criamos colunas binárias para cada categoria.

In [ ]:
# One-Hot Encoding
print("=== ONE-HOT ENCODING ===")

print("\nANTES do One-Hot:")
print(f"Shape: {df.shape}")

# Aplicar One-Hot
df = pd.get_dummies(df,
                    columns=nominal_vars,
                    prefix=nominal_vars,
                    drop_first=True,  # Evitar multicolinearidade
                    dtype=int)

print("\nDEPOIS do One-Hot:")
print(f"Shape: {df.shape}")

# Listar novas colunas criadas
new_cols = [c for c in df.columns if any(nom in c for nom in nominal_vars)]
print(f"\nColunas criadas ({len(new_cols)}):")
for col in new_cols:
    print(f"  - {col}")

### Por que drop_first=True?

Com 3 categorias de InternetService:
- Se DSL=0 e Fiber=0, então necessariamente No=1
- Informação redundante (multicolinearidade perfeita)
- drop_first=True remove primeira categoria (vira referência)

In [ ]:
# Demonstrar drop_first
print("=== DEMONSTRAÇÃO: drop_first ===")

# Recarregar dados para exemplo
df_exemplo = pd.read_csv('telco_churn_with_features.csv')

# Sem drop_first
dummy_full = pd.get_dummies(df_exemplo['InternetService'], prefix='Internet', drop_first=False)
print("\nSEM drop_first:")
print(f"Colunas: {dummy_full.columns.tolist()}")
print(dummy_full.head())

# Com drop_first
dummy_dropped = pd.get_dummies(df_exemplo['InternetService'], prefix='Internet', drop_first=True)
print("\nCOM drop_first:")
print(f"Colunas: {dummy_dropped.columns.tolist()}")
print(dummy_dropped.head())
print("\nCategoria de referência (implícita): DSL")

## Verificação Final: Encoding Completo

In [ ]:
# Verificar se ainda há categóricas
print("=== VERIFICAÇÃO: ENCODING COMPLETO ===")

categorical_remaining = df.select_dtypes(include='object').columns.tolist()
categorical_remaining = [c for c in categorical_remaining if c not in ['customerID', 'Churn']]

print(f"\nColunas categóricas restantes (exceto IDs): {len(categorical_remaining)}")

if len(categorical_remaining) == 0:
    print("✓ SUCESSO! Todas as features foram encodadas.")
else:
    print(f"⚠ ATENÇÃO: Ainda há colunas categóricas: {categorical_remaining}")

# Contar features numéricas
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
print(f"\nTotal de features numéricas: {len(numeric_features)}")

## Análise: Correlações com Dados Encodados

In [ ]:
# Matriz de correlação com features encodadas
print("=== MATRIZ DE CORRELAÇÃO (SUBSET) ===")

# Selecionar features importantes
analysis_features = [
    'tenure', 'MonthlyCharges', 'TotalCharges',
    'Contract_encoded', 'TotalServices',
    'InternetService_Fiber optic', 'InternetService_No',
    'PaymentMethod_Electronic check',
    'Partner_encoded', 'Dependents_encoded',
    'Churn_binary'
]

# Calcular correlações
corr_matrix = df[analysis_features].corr()

# Heatmap
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5, ax=ax,
            cbar_kws={'label': 'Correlação'})
ax.set_title('Matriz de Correlação - Features Encodadas', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Top correlações com Churn
correlations = corr_matrix['Churn_binary'].drop('Churn_binary').sort_values(key=abs, ascending=False)

print("=== TOP 10 CORRELAÇÕES COM CHURN ===")
print(correlations.head(10))

# Visualizar
fig, ax = plt.subplots(figsize=(10, 6))
correlations.head(10).plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Correlação')
ax.set_title('Top 10 Features Correlacionadas com Churn', fontsize=14)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n✓ Encoding completo! Próximo: Normalização")

---

# BLOCO 3: Normalização e Padronização

**Objetivo:** Colocar variáveis em escalas comparáveis para facilitar análise e visualização.

**Técnicas:**
1. **Min-Max Scaling** - Escala para [0, 1]
2. **Standardization (Z-score)** - Média 0, Desvio Padrão 1
3. **Robust Scaling** - Resistente a outliers

---

## Problema: Escalas Diferentes

In [ ]:
# Visualizar problema
print("=== PROBLEMA: ESCALAS DIFERENTES ===")

comparison_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']

for var in comparison_vars:
    print(f"\n{var}:")
    print(f"  Min: {df[var].min():.2f}")
    print(f"  Max: {df[var].max():.2f}")
    print(f"  Range: {df[var].max() - df[var].min():.2f}")

In [ ]:
# Visualizar o problema
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot mostra escalas incomparáveis
df[comparison_vars].boxplot(ax=axes[0])
axes[0].set_title('ANTES: Escalas Incomparáveis', fontsize=14)
axes[0].set_ylabel('Valores')

# Scatter mostra dominância de uma variável
axes[1].scatter(df['tenure'], df['TotalCharges'], alpha=0.3)
axes[1].set_xlabel('tenure (0-72)')
axes[1].set_ylabel('TotalCharges (0-8684)')
axes[1].set_title('Escalas Muito Diferentes', fontsize=14)

plt.tight_layout()
plt.show()

print("\n⚠ Problema: TotalCharges domina visualmente por ter range maior")
print("Mas isso NÃO significa que é mais importante!")

## Técnica 1: Min-Max Scaling (Normalização)

**Fórmula:** $X_{scaled} = \frac{X - X_{min}}{X_{max} - X_{min}}$

**Resultado:** Todos os valores entre 0 e 1

In [ ]:
# Min-Max Scaling
print("=== MIN-MAX SCALING ===")

scaler_minmax = MinMaxScaler()

# Selecionar colunas numéricas para normalizar
numeric_cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Aplicar
df[[f'{col}_minmax' for col in numeric_cols_to_scale]] = scaler_minmax.fit_transform(
    df[numeric_cols_to_scale]
)

print("\n✓ Min-Max Scaling aplicado")

# Comparar ANTES e DEPOIS
print("\n=== ANTES (original) ===")
print(df[numeric_cols_to_scale].describe())

print("\n=== DEPOIS (min-max) ===")
minmax_cols = [f'{col}_minmax' for col in numeric_cols_to_scale]
print(df[minmax_cols].describe())

In [ ]:
# Visualizar transformação
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, col in enumerate(numeric_cols_to_scale):
    # Original
    axes[0, idx].hist(df[col], bins=30, color='skyblue', edgecolor='black')
    axes[0, idx].set_title(f'Original: {col}')
    axes[0, idx].set_xlabel(col)

    # Normalizado
    axes[1, idx].hist(df[f'{col}_minmax'], bins=30, color='coral', edgecolor='black')
    axes[1, idx].set_title(f'Min-Max: {col}')
    axes[1, idx].set_xlabel(f'{col} (0-1)')

plt.tight_layout()
plt.show()

print("\n✓ Forma da distribuição preservada, mas escala agora é [0, 1]")

## Técnica 2: Standardization (Z-score)

**Fórmula:** $X_{scaled} = \frac{X - \mu}{\sigma}$

**Resultado:** Média = 0, Desvio Padrão = 1

In [ ]:
# Standardization
print("=== STANDARDIZATION (Z-SCORE) ===")

scaler_standard = StandardScaler()

# Aplicar
df[[f'{col}_std' for col in numeric_cols_to_scale]] = scaler_standard.fit_transform(
    df[numeric_cols_to_scale]
)

print("\n✓ Standardization aplicado")

# Verificar resultado
print("\n=== DEPOIS (standardized) ===")
std_cols = [f'{col}_std' for col in numeric_cols_to_scale]
print(df[std_cols].describe())

print("\n✓ Média ≈ 0, Desvio Padrão ≈ 1 (conforme esperado)")

In [ ]:
# Interpretação de Z-scores
print("=== INTERPRETAÇÃO DE Z-SCORES ===")
print("\nExemplo: tenure_std")

tenure_std = df['tenure_std']

print(f"\nPercentual dentro de ranges:")
print(f"  Entre -1 e +1 (±1 std): {((tenure_std >= -1) & (tenure_std <= 1)).mean()*100:.1f}%")
print(f"  Entre -2 e +2 (±2 std): {((tenure_std >= -2) & (tenure_std <= 2)).mean()*100:.1f}%")
print(f"  Entre -3 e +3 (±3 std): {((tenure_std >= -3) & (tenure_std <= 3)).mean()*100:.1f}%")

outliers = (abs(tenure_std) > 3).sum()
print(f"\nOutliers (|Z| > 3): {outliers} clientes ({outliers/len(df)*100:.2f}%)")

In [ ]:
# Visualizar Z-scores
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, col in enumerate(numeric_cols_to_scale):
    std_col = f'{col}_std'

    axes[idx].hist(df[std_col], bins=30, color='lightgreen', edgecolor='black')
    axes[idx].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Média (0)')
    axes[idx].axvline(x=-1, color='orange', linestyle='--', alpha=0.5, label='±1 std')
    axes[idx].axvline(x=1, color='orange', linestyle='--', alpha=0.5)
    axes[idx].set_title(f'Z-score: {col}')
    axes[idx].set_xlabel('Z-score')
    axes[idx].legend()

plt.tight_layout()
plt.show()

## Técnica 3: Robust Scaling

**Fórmula:** $X_{scaled} = \frac{X - median}{IQR}$

**Vantagem:** Resistente a outliers (usa mediana e IQR)

In [ ]:
# Robust Scaling
print("=== ROBUST SCALING ===")

scaler_robust = RobustScaler()

# Aplicar
df[[f'{col}_robust' for col in numeric_cols_to_scale]] = scaler_robust.fit_transform(
    df[numeric_cols_to_scale]
)

print("\n✓ Robust Scaling aplicado")

# Verificar resultado
print("\n=== DEPOIS (robust) ===")
robust_cols = [f'{col}_robust' for col in numeric_cols_to_scale]
print(df[robust_cols].describe())

print("\n✓ Mediana ≈ 0, escala baseada em IQR")

## Comparação: Min-Max vs Standard vs Robust

In [ ]:
# Comparar as três técnicas
print("=== COMPARAÇÃO DAS TÉCNICAS (tenure) ===\n")

scaling_methods = {
    'Original': 'tenure',
    'Min-Max': 'tenure_minmax',
    'Standard': 'tenure_std',
    'Robust': 'tenure_robust'
}

comparison_df = pd.DataFrame()
for name, col in scaling_methods.items():
    comparison_df[name] = [
        df[col].min(),
        df[col].quantile(0.25),
        df[col].median(),
        df[col].mean(),
        df[col].quantile(0.75),
        df[col].max(),
        df[col].std()
    ]

comparison_df.index = ['Min', 'Q1', 'Median', 'Mean', 'Q3', 'Max', 'Std']
print(comparison_df)

In [ ]:
# Visualizar comparação lado a lado
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, (name, col) in enumerate(scaling_methods.items()):
    axes[idx].hist(df[col], bins=30, color=['skyblue', 'coral', 'lightgreen', 'orchid'][idx],
                   edgecolor='black')
    axes[idx].set_title(name, fontsize=12)
    axes[idx].set_xlabel(col)
    axes[idx].axvline(x=df[col].mean(), color='red', linestyle='--', alpha=0.7, label='Mean')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("\n✓ Mesma distribuição, escalas diferentes")

## Comparação Visual: ANTES vs DEPOIS

In [ ]:
# Boxplot comparativo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ANTES: escalas incomparáveis
df[numeric_cols_to_scale].boxplot(ax=axes[0])
axes[0].set_title('ANTES: Escalas Incomparáveis', fontsize=14)
axes[0].set_ylabel('Valores Originais')

# DEPOIS: escalas comparáveis (usando Standard)
df[std_cols].boxplot(ax=axes[1])
axes[1].set_title('DEPOIS: Escalas Comparáveis (Z-score)', fontsize=14)
axes[1].set_ylabel('Z-score')
axes[1].set_xticklabels(['tenure', 'MonthlyCharges', 'TotalCharges'])
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n✓ Agora podemos comparar variáveis diretamente!")

## Boa Prática: Separar Dados para Validação

**Importante:** Em cenários reais, devemos treinar o scaler apenas nos dados de treino e aplicar aos dados de teste.

In [ ]:
# Demonstração de fit vs transform
print("=== BOA PRÁTICA: FIT vs TRANSFORM ===")

# Separar dados (80% treino, 20% teste)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

print(f"\nTreino: {len(df_train)} registros")
print(f"Teste: {len(df_test)} registros")

# Criar scaler
scaler_demo = StandardScaler()

# FIT apenas no treino
scaler_demo.fit(df_train[numeric_cols_to_scale])

print("\n=== PARÂMETROS APRENDIDOS (do treino) ===")
print(f"Médias: {scaler_demo.mean_}")
print(f"Desvios: {scaler_demo.scale_}")

In [ ]:
# TRANSFORM em ambos (usando mesmos parâmetros)
train_scaled = scaler_demo.transform(df_train[numeric_cols_to_scale])
test_scaled = scaler_demo.transform(df_test[numeric_cols_to_scale])

# Verificar
print("=== VERIFICAÇÃO ===")
print("\nConjunto TREINO:")
print(f"  Média tenure: {train_scaled[:, 0].mean():.3f} (≈ 0)")
print(f"  Std tenure: {train_scaled[:, 0].std():.3f} (≈ 1)")

print("\nConjunto TESTE:")
print(f"  Média tenure: {test_scaled[:, 0].mean():.3f} (pode ≠ 0)")
print(f"  Std tenure: {test_scaled[:, 0].std():.3f} (pode ≠ 1)")

print("\n✓ Teste usa parâmetros do treino (correto!)")
print("Isso simula como seria com dados futuros em produção.")

## Resumo do Bloco 3

In [ ]:
# Resumo das técnicas
print("=== RESUMO: NORMALIZAÇÃO E PADRONIZAÇÃO ===")
print("\n1. MIN-MAX SCALING")
print("   - Range: [0, 1]")
print("   - Use quando: quer valores entre 0 e 1")
print("   - Sensível a: outliers")

print("\n2. STANDARDIZATION (Z-SCORE)")
print("   - Média: 0, Std: 1")
print("   - Use quando: quer centralizar em zero")
print("   - Interpretação: desvios padrão")

print("\n3. ROBUST SCALING")
print("   - Baseado em: mediana e IQR")
print("   - Use quando: muitos outliers")
print("   - Resistente a: valores extremos")

print("\n✓ Todas preservam a forma da distribuição")
print("✓ Apenas mudam a escala")
print("\n✓ Pronto para análises avançadas!")

---

**Checkpoint:** Salvar dataset com todas as transformações

In [ ]:
# Salvar dataset transformado
df.to_csv('telco_churn_transformed.csv', index=False)
print("✓ Dataset salvo: telco_churn_transformed.csv")
print(f"Shape final: {df.shape}")
print(f"\n✓ Transformações completas:")
print("  - Feature Engineering")
print("  - Encoding")
print("  - Normalização")
print("\nPróximo: Pipeline completo e análise final!")

# Aula 10 - PARTE 3: Pipeline Completo e Análise Final

**Integração de todas as transformações**

Nesta parte:
- **Bloco 4:** Pipeline end-to-end
- Análise exploratória avançada com dados transformados
- Conclusões e próximos passos

---

## Setup (caso esteja começando daqui)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

print("✓ Setup completo!")

In [ ]:
# Carregar dados originais (vamos aplicar pipeline do zero)
url = 'https://raw.githubusercontent.com/marvin-rubia/Churn-Analysis-Prediction/main/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df_original = pd.read_csv(url)


# Tratamento básico
df_original['TotalCharges'] = pd.to_numeric(df_original['TotalCharges'], errors='coerce')
df_original['TotalCharges'].fillna(df_original['MonthlyCharges'], inplace=True)

print(f"Dataset original: {df_original.shape}")
print("✓ Pronto para aplicar pipeline completo")

---

# BLOCO 4: Pipeline Completo

**Objetivo:** Criar função única que aplica todas as transformações de uma vez.

**Benefícios:**
- Reproduzível
- Organizado
- Fácil de aplicar em dados novos
- Evita erros de esquecimento de etapas

---

## Criar Função Pipeline Completa

In [ ]:
def transform_telco_churn(df, fit_scaler=True, scaler=None):
    """
    Pipeline completo de transformação para Telco Churn Dataset

    Aplica:
    1. Feature Engineering
    2. Encoding de categóricas
    3. Normalização (Standardization)

    Parameters:
    -----------
    df : DataFrame
        Dataset bruto
    fit_scaler : bool
        Se True, fit novo scaler. Se False, usa scaler fornecido
    scaler : sklearn scaler, optional
        Scaler pré-treinado (para aplicar em dados novos)

    Returns:
    --------
    df_transformed : DataFrame
        Dataset transformado
    scaler : StandardScaler
        Scaler treinado (para usar em dados futuros)
    """

    print("=== INICIANDO PIPELINE DE TRANSFORMAÇÃO ===")
    df_transformed = df.copy()

    # ===== 1. FEATURE ENGINEERING =====
    print("\n1. Feature Engineering...")

    # Operações aritméticas
    df_transformed['AvgMonthlySpend'] = df_transformed['TotalCharges'] / df_transformed['tenure']
    df_transformed['AvgMonthlySpend'] = df_transformed['AvgMonthlySpend'].replace([np.inf, -np.inf], np.nan)
    df_transformed.loc[df_transformed['tenure'] == 0, 'AvgMonthlySpend'] = \
        df_transformed.loc[df_transformed['tenure'] == 0, 'MonthlyCharges']

    # Agregações
    service_cols = ['PhoneService', 'MultipleLines', 'InternetService',
                   'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                   'TechSupport', 'StreamingTV', 'StreamingMovies']

    df_transformed['TotalServices'] = 0
    for col in service_cols:
        df_transformed['TotalServices'] += (
            ~df_transformed[col].isin(['No', 'No internet service', 'No phone service'])
        ).astype(int)

    # Flags
    df_transformed['IsPremium'] = (
        (df_transformed['TotalServices'] >= 6) &
        (df_transformed['Contract'].isin(['One year', 'Two year']))
    ).astype(int)

    df_transformed['HighRiskProfile'] = (
        (df_transformed['Contract'] == 'Month-to-month') &
        (df_transformed['tenure'] < 12) &
        (df_transformed['TotalServices'] <= 2) &
        (df_transformed['PaymentMethod'] == 'Electronic check')
    ).astype(int)

    print("   ✓ Features criadas: AvgMonthlySpend, TotalServices, IsPremium, HighRiskProfile")

    # ===== 2. ENCODING =====
    print("\n2. Encoding...")

    # Ordinais (Label)
    contract_map = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
    df_transformed['Contract_encoded'] = df_transformed['Contract'].map(contract_map)

    # Binárias
    binary_vars = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
    for col in binary_vars:
        if col == 'gender':
            df_transformed[f'{col}_encoded'] = (df_transformed[col] == 'Male').astype(int)
        else:
            df_transformed[f'{col}_encoded'] = (df_transformed[col] == 'Yes').astype(int)

    df_transformed['SeniorCitizen_encoded'] = df_transformed['SeniorCitizen']

    # Serviços com "No service"
    service_with_no = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup',
                       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    for col in service_with_no:
        df_transformed[f'{col}_encoded'] = (df_transformed[col] == 'Yes').astype(int)

    # Nominais (One-Hot)
    nominal_vars = ['PaymentMethod', 'InternetService']
    df_transformed = pd.get_dummies(df_transformed,
                                    columns=nominal_vars,
                                    prefix=nominal_vars,
                                    drop_first=True,
                                    dtype=int)

    print("   ✓ Encoding completo: Label, Binary, One-Hot")

    # ===== 3. NORMALIZAÇÃO =====
    print("\n3. Normalização (Standardization)...")

    # Colunas para normalizar
    numeric_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges',
                        'AvgMonthlySpend', 'TotalServices']

    if fit_scaler:
        # Criar e treinar novo scaler
        scaler = StandardScaler()
        df_transformed[numeric_to_scale] = scaler.fit_transform(df_transformed[numeric_to_scale])
        print("   ✓ Scaler treinado e aplicado")
    else:
        # Usar scaler existente
        if scaler is None:
            raise ValueError("Forneça scaler quando fit_scaler=False")
        df_transformed[numeric_to_scale] = scaler.transform(df_transformed[numeric_to_scale])
        print("   ✓ Scaler existente aplicado")

    # Target binário
    df_transformed['Churn_binary'] = (df_transformed['Churn'] == 'Yes').astype(int)

    print("\n=== PIPELINE COMPLETO! ===")
    print(f"Shape: {df_original.shape} → {df_transformed.shape}")

    return df_transformed, scaler

## Aplicar Pipeline ao Dataset

In [ ]:
# Aplicar pipeline
df_final, trained_scaler = transform_telco_churn(df_original, fit_scaler=True)

print("\n" + "="*50)
print("TRANSFORMAÇÃO COMPLETA!")
print("="*50)

## Validar Pipeline

In [ ]:
# Checklist de validação
print("=== CHECKLIST DE VALIDAÇÃO ===")

# 1. Missing values
missing = df_final.isnull().sum().sum()
print(f"\n1. Missing values: {missing}")
if missing == 0:
    print("   ✓ Nenhum missing value")
else:
    print(f"   ⚠ {missing} missing values encontrados")

# 2. Variáveis categóricas restantes
cat_cols = df_final.select_dtypes('object').columns.tolist()
cat_cols = [c for c in cat_cols if c not in ['customerID', 'Churn']]
print(f"\n2. Categóricas restantes (exceto IDs): {len(cat_cols)}")
if len(cat_cols) == 0:
    print("   ✓ Todas encodadas")
else:
    print(f"   ⚠ Ainda há categóricas: {cat_cols}")

# 3. Features numéricas
numeric_features = df_final.select_dtypes(include=['int64', 'float64']).columns
print(f"\n3. Features numéricas: {len(numeric_features)}")
print(f"   ✓ {len(numeric_features)} features prontas para análise")

# 4. Verificar normalização
print("\n4. Normalização (tenure, MonthlyCharges, TotalCharges):")
for col in ['tenure', 'MonthlyCharges', 'TotalCharges']:
    mean = df_final[col].mean()
    std = df_final[col].std()
    print(f"   {col}: média={mean:.3f}, std={std:.3f}")
print("   ✓ Média ≈ 0, Std ≈ 1 (normalizado)")

# 5. Target
print(f"\n5. Target (Churn_binary):")
print(f"   Distribuição: {df_final['Churn_binary'].value_counts().to_dict()}")
print(f"   Taxa de Churn: {df_final['Churn_binary'].mean()*100:.1f}%")
print("   ✓ Target criado corretamente")

print("\n" + "="*50)
print("✓ TODAS AS VALIDAÇÕES PASSARAM!")
print("="*50)

---

## Análise Exploratória Avançada

Agora que os dados estão completamente transformados, podemos fazer análises que não eram possíveis antes.

---

### 1. Matriz de Correlação Completa

In [ ]:
# Selecionar features importantes para análise
analysis_features = [
    # Numéricas originais (normalizadas)
    'tenure', 'MonthlyCharges', 'TotalCharges',
    # Features criadas
    'AvgMonthlySpend', 'TotalServices',
    # Encodadas
    'Contract_encoded', 'IsPremium', 'HighRiskProfile',
    'SeniorCitizen_encoded', 'Partner_encoded', 'Dependents_encoded',
    # One-Hot (exemplos)
    'InternetService_Fiber optic', 'InternetService_No',
    'PaymentMethod_Electronic check',
    # Target
    'Churn_binary'
]

# Calcular matriz de correlação
corr_matrix = df_final[analysis_features].corr()

# Heatmap
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=0.5,
            cbar_kws={'label': 'Correlação'}, ax=ax)
ax.set_title('Matriz de Correlação - Dataset Transformado', fontsize=16)
plt.tight_layout()
plt.show()

print("✓ Matriz de correlação com todas as features transformadas")

### 2. Análise de Importância das Features

In [ ]:
# Top correlações com Churn
print("=== TOP 20 FEATURES MAIS CORRELACIONADAS COM CHURN ===")

correlations_with_churn = corr_matrix['Churn_binary'].drop('Churn_binary')
top_20 = correlations_with_churn.abs().sort_values(ascending=False).head(20)

print("\nFeature | Correlação")
print("-" * 50)
for feat, corr in top_20.items():
    actual_corr = correlations_with_churn[feat]
    direction = "↑ aumenta" if actual_corr > 0 else "↓ reduz"
    print(f"{feat:35} | {actual_corr:+.3f} ({direction} churn)")

In [ ]:
# Visualizar top correlações
fig, ax = plt.subplots(figsize=(10, 8))

# Separar positivas e negativas
top_corr = correlations_with_churn.abs().sort_values(ascending=False).head(15)
values = [correlations_with_churn[feat] for feat in top_corr.index]
colors = ['salmon' if v > 0 else 'skyblue' for v in values]

ax.barh(range(len(top_corr)), values, color=colors)
ax.set_yticks(range(len(top_corr)))
ax.set_yticklabels(top_corr.index)
ax.set_xlabel('Correlação com Churn', fontsize=12)
ax.set_title('Top 15 Features por Correlação com Churn', fontsize=14)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')

# Legenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='salmon', label='Positiva (↑ churn)'),
    Patch(facecolor='skyblue', label='Negativa (↓ churn)')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

### 3. Insights de Negócio

In [ ]:
# Extrair insights principais
print("=== INSIGHTS DE NEGÓCIO ===")
print("\nFatores que AUMENTAM churn (correlação positiva):")
print("-" * 60)

positive_corr = correlations_with_churn[correlations_with_churn > 0].sort_values(ascending=False)
for feat, corr in positive_corr.head(5).items():
    print(f"  • {feat}: {corr:.3f}")

print("\nFatores que REDUZEM churn (correlação negativa):")
print("-" * 60)

negative_corr = correlations_with_churn[correlations_with_churn < 0].sort_values()
for feat, corr in negative_corr.head(5).items():
    print(f"  • {feat}: {corr:.3f}")

In [ ]:
# Recomendações baseadas nos dados
print("\n" + "="*60)
print("RECOMENDAÇÕES PARA REDUZIR CHURN")
print("="*60)

print("\n1. FOCO EM CONTRATOS LONGOS")
print("   • Contract_encoded tem correlação negativa forte (-0.40+)")
print("   • Ação: Incentivar migração de month-to-month para anual")
print("   • Estratégia: Descontos para contratos de 1-2 anos")

print("\n2. AUMENTAR ENGAJAMENTO (SERVIÇOS)")
print("   • TotalServices correlaciona negativamente com churn")
print("   • Ação: Cross-sell de serviços adicionais")
print("   • Estratégia: Bundles atrativos de múltiplos serviços")

print("\n3. ATENÇÃO A PERFIS DE ALTO RISCO")
print("   • HighRiskProfile identifica clientes vulneráveis")
print("   • Ação: Programa de retenção proativa")
print("   • Estratégia: Contato preventivo antes do churn")

print("\n4. REVISAR FIBRA ÓPTICA")
print("   • InternetService_Fiber optic aumenta churn")
print("   • Possível causa: Preço vs expectativa")
print("   • Ação: Investigar satisfação e ajustar preço/qualidade")

print("\n5. MÉTODO DE PAGAMENTO")
print("   • Electronic check associado a mais churn")
print("   • Ação: Incentivar métodos mais estáveis (débito automático)")
print("   • Estratégia: Descontos para débito automático")

### 4. Comparação: Antes vs Depois das Transformações

In [ ]:
# Visualização lado a lado
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. ANTES: Variáveis em escalas diferentes
ax1 = fig.add_subplot(gs[0, 0])
df_original[['tenure', 'MonthlyCharges', 'TotalCharges']].boxplot(ax=ax1)
ax1.set_title('ANTES: Escalas Incomparáveis', fontsize=12, fontweight='bold')
ax1.set_ylabel('Valores Originais')

# 2. DEPOIS: Variáveis normalizadas
ax2 = fig.add_subplot(gs[0, 1])
df_final[['tenure', 'MonthlyCharges', 'TotalCharges']].boxplot(ax=ax2)
ax2.set_title('DEPOIS: Escalas Comparáveis', fontsize=12, fontweight='bold')
ax2.set_ylabel('Z-score')
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# 3. ANTES: Poucas features numéricas
ax3 = fig.add_subplot(gs[1, 0])
original_numeric = df_original.select_dtypes(include=['int64', 'float64']).columns
ax3.text(0.5, 0.5, f'Features Numéricas\nOriginais: {len(original_numeric)}\n\n' +
         '\n'.join(original_numeric[:10].tolist()),
         ha='center', va='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')
ax3.set_title('ANTES: Poucas Features', fontsize=12, fontweight='bold')

# 4. DEPOIS: Muitas features numéricas
ax4 = fig.add_subplot(gs[1, 1])
final_numeric = df_final.select_dtypes(include=['int64', 'float64']).columns
ax4.text(0.5, 0.5, f'Features Numéricas\nFinais: {len(final_numeric)}\n\n' +
         f'+ {len(final_numeric) - len(original_numeric)} features criadas!',
         ha='center', va='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')
ax4.set_title('DEPOIS: Muitas Features', fontsize=12, fontweight='bold')

# 5. ANTES: Apenas 3 correlações possíveis
ax5 = fig.add_subplot(gs[2, 0])
original_corr = df_original[['tenure', 'MonthlyCharges', 'TotalCharges']].corr()
sns.heatmap(original_corr, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, ax=ax5, cbar=False)
ax5.set_title('ANTES: Correlações Limitadas', fontsize=12, fontweight='bold')

# 6. DEPOIS: Análise rica de correlações
ax6 = fig.add_subplot(gs[2, 1])
subset_features = ['tenure', 'MonthlyCharges', 'Contract_encoded',
                   'TotalServices', 'IsPremium', 'Churn_binary']
final_corr = df_final[subset_features].corr()
sns.heatmap(final_corr, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, ax=ax6, cbar=False)
ax6.set_title('DEPOIS: Análise Rica', fontsize=12, fontweight='bold')

plt.suptitle('IMPACTO DAS TRANSFORMAÇÕES', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("✓ Transformações permitiram análises que eram impossíveis antes!")

### 5. Perfis de Clientes Transformados

In [ ]:
# Analisar perfis criados
print("=== ANÁLISE DE PERFIS ===")

profiles = ['IsPremium', 'HighRiskProfile']

for profile in profiles:
    print(f"\n{'='*60}")
    print(f"{profile}")
    print('='*60)

    # Quantidade
    count = df_final[profile].sum()
    pct = df_final[profile].mean() * 100
    print(f"\nQuantidade: {count} clientes ({pct:.1f}% da base)")

    # Churn rate
    churn_rate_with = df_final[df_final[profile] == 1]['Churn_binary'].mean()
    churn_rate_without = df_final[df_final[profile] == 0]['Churn_binary'].mean()
    baseline = df_final['Churn_binary'].mean()

    print(f"\nTaxa de Churn:")
    print(f"  Com {profile}=1: {churn_rate_with*100:.1f}%")
    print(f"  Com {profile}=0: {churn_rate_without*100:.1f}%")
    print(f"  Baseline (geral): {baseline*100:.1f}%")
    print(f"  Diferença: {(churn_rate_with - baseline)*100:+.1f} pontos percentuais")

    # Características médias
    print(f"\nCaracterísticas médias (com {profile}=1):")
    subset = df_final[df_final[profile] == 1]
    print(f"  Tenure (meses): {subset['tenure'].mean():.2f} (original: {df_original[df_original.index.isin(subset.index)]['tenure'].mean():.0f})")
    print(f"  MonthlyCharges: R$ {df_original[df_original.index.isin(subset.index)]['MonthlyCharges'].mean():.2f}")
    print(f"  TotalServices: {df_original[df_original.index.isin(subset.index)]['TotalServices'].mean():.1f}" if 'TotalServices' in df_original.columns else "")

---

## Conclusão e Próximos Passos

---

### O que Aprendemos Hoje

In [ ]:
print("="*70)
print("RESUMO DA AULA 10: PRÉ-PROCESSAMENTO DE DADOS")
print("="*70)

print("\n1. FEATURE ENGINEERING")
print("   ✓ Criamos variáveis derivadas úteis")
print("   ✓ 12 técnicas diferentes aplicadas")
print("   ✓ Validação através de correlações e visualizações")
print(f"   → Resultado: {len([c for c in df_final.columns if c not in df_original.columns])} novas features")

print("\n2. ENCODING")
print("   ✓ Convertemos todas as categóricas em numéricas")
print("   ✓ Label (ordinais), One-Hot (nominais), Binary (sim/não)")
print("   ✓ Estratégia apropriada para cada tipo")
original_categorical = len(df_original.select_dtypes('object').columns) - 2  # -2 for ID and target
print(f"   → Resultado: {original_categorical} variáveis categóricas encodadas")

print("\n3. NORMALIZAÇÃO")
print("   ✓ Colocamos variáveis em escalas comparáveis")
print("   ✓ Min-Max, Standardization, Robust Scaling")
print("   ✓ Escolha da técnica apropriada")
print("   → Resultado: Todas as variáveis comparáveis (Z-score)")

print("\n4. PIPELINE COMPLETO")
print("   ✓ Função reproduzível end-to-end")
print("   ✓ Aplicável a dados novos")
print("   ✓ Organizado e documentado")
print("   → Resultado: Pipeline pronto para produção")

print("\n" + "="*70)
print("IMPACTO")
print("="*70)
print(f"Shape: {df_original.shape} → {df_final.shape}")
print(f"Features numéricas: {len(df_original.select_dtypes('number').columns)} → {len(df_final.select_dtypes('number').columns)}")
print(f"Análises possíveis: BÁSICAS → AVANÇADAS")
print("\n✓ DADOS PRONTOS PARA ANÁLISES SOFISTICADAS!")

### Conexão com Machine Learning (Preview)

In [ ]:
print("="*70)
print("PREVIEW: CONEXÃO COM MACHINE LEARNING")
print("="*70)

print("\nO QUE FIZEMOS HOJE:")
print("  • Feature Engineering → criar variáveis úteis")
print("  • Encoding → converter para números")
print("  • Normalização → escalas comparáveis")
print("  • OBJETIVO: Análise exploratória avançada")

print("\nEM MACHINE LEARNING (CURSOS FUTUROS):")
print("  • Usaremos EXATAMENTE o mesmo pipeline")
print("  • Mas ao invés de análise manual...")
print("  • Treinaremos algoritmos para aprender padrões automaticamente")
print("  • Ex: Algoritmo aprende que 'Contract_encoded=2 + TotalServices≥6 → baixo churn'")
print("  • Depois aplica em clientes novos para prever risco")

print("\n✓ Você já domina a FUNDAÇÃO essencial para ML!")
print("✓ Mesmas transformações, objetivos diferentes")
print("✓ Quando chegar em ML, já saberá preparar dados corretamente")

print("\n" + "="*70)

### Salvar Resultados

In [ ]:
# Salvar dataset final
df_final.to_csv('telco_churn_final_transformed.csv', index=False)
print("✓ Dataset final salvo: telco_churn_final_transformed.csv")

# Salvar scaler (para usar em dados futuros)
import joblib
joblib.dump(trained_scaler, 'scaler_telco_churn.pkl')
print("✓ Scaler salvo: scaler_telco_churn.pkl")

print("\n✓ Todos os artefatos salvos!")
print("\nArquivos gerados:")
print("  • telco_churn_final_transformed.csv - Dataset transformado")
print("  • scaler_telco_churn.pkl - Scaler treinado")